In [7]:
!conda install pytorch torchvision -c pytorch

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    torchvision: 0.2.1-py35_0 --> 0.2.1-py35_1 pytorch



In [8]:
#https://github.com/pytorch/examples/blob/master/mnist/main.py

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import time as time

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        
        # input is 28x28x1
        # conv1(kernel=5, filters=10) 28x28x10 -> 24x24x10
        # max_pool(kernel=2) 24x24x10 -> 12x12x10
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        
        # conv2(kernel=5, filters=20) 12x12x20 -> 8x8x20
        # max_pool(kernel=2) 8x8x20 -> 4x4x20
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        
        # flatten 4x4x20 = 320
        x = x.view(-1, 320)
        # 320 -> 50
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        # 50 -> 10
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


In [10]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [19]:
def main():
    # Training settings
    batch_size = 128
    test_batch_size = 1000
    epochs = 2
    lr = 0.05
    momentum = 0.5
    seed = 42
    
    use_cuda = False
    torch.manual_seed(seed)
    device = torch.device("cuda" if use_cuda else "cpu")
      
    # download and transform train dataset
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])), batch_size = batch_size, shuffle=True, num_workers=4)
    
    # download and transform test dataset
    test_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))
                       ])), batch_size = test_batch_size, shuffle=True, num_workers=4)


    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    for epoch in range(1, epochs + 1):
        start = time.time()
        train(model, device, train_loader, optimizer, epoch)
        print('Training took {:.3f} s'.format((time.time()-start)))
        test(model, device, test_loader)


if __name__ == '__main__':
    main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.307264
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.648696
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.526176
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.470979
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.291871
Training took 144.115 s

Test set: Average loss: 0.1250, Accuracy: 9621/10000 (96%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.235891
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.316604
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.331153
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.374789
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.242344
Training took 145.230 s

Test set: Average loss: 0.0815, Accuracy: 9756/10000 (98%)

